<h1>Setup</h1>


In [ ]:
!pip install gymnasium[box2d] shimmy
!pip install imageio imageio-ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.3 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
import imageio
import os
import sys
from IPython.display import Video

<h1>Load Expert</h1>

In [ ]:
# Load the trained Keras model
MODEL_PATH = "model_10.keras"

try:
    model = keras.models.load_model(MODEL_PATH)
    print(f"Model '{MODEL_PATH}' loaded successfully!")
except:
    print(f"Error: File '{MODEL_PATH}' not found. Please upload it.")
    sys.exit(1)


env = gym.make("LunarLander-v3", render_mode="rgb_array")

Model 'model_10.keras' loaded successfully!


<h1>Reward Check</h1>

In [9]:
def evaluate_model(model, env, n_episodes=10):

    print(f"Starting evaluation over {n_episodes} episodes...")
    total_rewards = []

    for i in range(n_episodes):
        obs, _ = env.reset()
        done = False
        episode_reward = 0
        step_count = 0

        while not done:
            # Prepare input for the model
            obs_input = np.expand_dims(obs, axis=0)

            # Predict action probabilities
            # verbose=0 prevents Keras from printing a progress bar for every single step
            action_probs = model.predict(obs_input, verbose=0)

            # Choose the action with the highest probability (Greedy strategy)
            action = np.argmax(action_probs[0])

            # Execute the action in the environment
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode_reward += reward
            step_count += 1

        total_rewards.append(episode_reward)

        # Print progress after each episode
        print(f"Episode {i+1}/{n_episodes} finished | Reward: {episode_reward:.2f} | Steps: {step_count}")

    # Calculate statistics
    mean_reward = np.mean(total_rewards)
    std_reward = np.std(total_rewards)

    print("-" * 30)
    print(f"Final Results:")
    print(f"Mean Reward: {mean_reward:.2f}")
    print(f"Standard Deviation: {std_reward:.2f}")




evaluate_model(model, env, n_episodes=10)

Starting evaluation over 10 episodes...
Episode 1/10 finished | Reward: 285.39 | Steps: 273
Episode 2/10 finished | Reward: 250.34 | Steps: 275
Episode 3/10 finished | Reward: 285.78 | Steps: 256
Episode 4/10 finished | Reward: 282.66 | Steps: 286
Episode 5/10 finished | Reward: 260.82 | Steps: 237
Episode 6/10 finished | Reward: 255.49 | Steps: 282
Episode 7/10 finished | Reward: 248.54 | Steps: 311
Episode 8/10 finished | Reward: 259.34 | Steps: 266
Episode 9/10 finished | Reward: 265.36 | Steps: 272
Episode 10/10 finished | Reward: 255.23 | Steps: 273
------------------------------
Final Results:
Mean Reward: 264.90
Standard Deviation: 13.71


<h1>The Video</h1>

In [12]:

def record_video(model, env, filename='lunar_lander_video.mp4'):
    print(" Recording video...")

    # Create a new environment specifically for recording
    video_env = gym.make("LunarLander-v3", render_mode="rgb_array")
    obs, _ = video_env.reset()
    done = False

    writer = imageio.get_writer(filename, fps=30)

    while not done:
        # Capture frame
        frame = video_env.render()
        writer.append_data(frame)

        # Get action
        obs_input = np.expand_dims(obs, axis=0)
        action = np.argmax(model.predict(obs_input, verbose=0)[0])

        # Step
        obs, _, terminated, truncated, _ = video_env.step(action)
        done = terminated or truncated

    writer.close()
    video_env.close()
    print(f" Video saved as '{filename}'.")

def show_video(filename='lunar_lander_video.mp4'):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Video file '{filename}' not found.")

    print(f"Displaying video: {filename}")
    display(Video(filename, embed=True))


record_video(model, env)
show_video()


 Recording video...
 Video saved as 'lunar_lander_video.mp4'.
Displaying video: lunar_lander_video.mp4
